In [1]:
import os
import shutil

import cv2
import numpy as np
from joblib import Parallel, delayed

In [2]:
def mask_file_to_yolo(mask_path: str, yolo_label_path: str):
    # Load or generate your segmentation mask
    # Assume mask is a 512x512 NumPy array where each pixel is an integer label
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    object_ids = np.unique(mask[mask > 0])
    
    with open(yolo_label_path, 'w') as f:
        # Find contours for the object with all labels
        for object_id in object_ids:
            label = object_id
            contours, _ = cv2.findContours((mask == label).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Generate the line for each contour found (in case there are multiple instances of the label)
            lines = []
            for contour in contours:
                # Flatten contour points and scale them back to [0, 1] relative coordinates
                contour_points = contour.reshape(-1, 2)
                contour_points = contour_points / 512  # Scale points to (0-1) range if required by YOLO
                points = ' '.join(f'{x:.6f} {y:.6f}' for x, y in contour_points)
                
                # Format: <class-index> <x1> <y1> <x2> <y2> ... <xn> <yn>
                line = f"{label - 1} {points}"
                lines.append(line)
        
            # Print result
            for line in lines:
                f.write(line + '\n')
                

In [4]:
for dataset in ['train', 'val']:
    image_path = f'./data/Public_leaderboard_data/{dataset}_images'
    yolo_image_path = f'./data/leaderboard_yolo_raw/images/{dataset}'
    os.makedirs(yolo_image_path, exist_ok=True)
    label_path = f'./data/Public_leaderboard_data/{dataset}_labels'
    yolo_label_path = f'./data/leaderboard_yolo_raw/labels/{dataset}'
    os.makedirs(yolo_label_path, exist_ok=True)
    
    print('Processing', dataset, 'set.')
    
    def parallel_convert(folder):
        for img in os.listdir(os.path.join(label_path, folder)):
            mask_file_to_yolo(os.path.join(label_path, folder, img), os.path.join(yolo_label_path, f'{folder}_{img.replace(".png", ".txt")}'))
            shutil.copy(os.path.join(image_path, folder, img), os.path.join(yolo_image_path, f'{folder}_{img}'))
            
            
    Parallel(n_jobs=-1)(delayed(parallel_convert)(folder) for folder in os.listdir(label_path))

Processing train set.
Processing val set.


In [12]:
for folder in ['images', 'labels']:
    for file in os.listdir(f'./data/leaderboard_yolo_ov/{folder}/val'):
        shutil.copy(os.path.join(f'./data/leaderboard_yolo_ov/{folder}/val', file), 
                    os.path.join(f'./data/leaderboard_yolo_ov/{folder}/train', file))
        # os.remove(os.path.join(f'./data/leaderboard_yolo/{folder}/train', file))
        print(file)

43_16.png
45_85.png
44_77.png
43_42.png
47_50.png
41_65.png
41_37.png
48_21.png
48_49.png
44_22.png
46_53.png
48_42.png
50_21.png
43_1.png
47_87.png
48_38.png
47_70.png
49_94.png
44_44.png
42_33.png
41_12.png
43_38.png
47_74.png
48_3.png
43_66.png
49_28.png
46_16.png
49_34.png
50_11.png
41_87.png
45_44.png
42_49.png
50_16.png
45_17.png
45_45.png
49_83.png
43_17.png
41_43.png
42_67.png
42_80.png
47_57.png
47_58.png
49_69.png
47_13.png
48_77.png
49_35.png
50_20.png
49_82.png
44_60.png
42_47.png
48_43.png
42_20.png
43_19.png
44_47.png
46_55.png
42_99.png
43_95.png
47_36.png
41_70.png
42_50.png
43_18.png
43_75.png
42_87.png
43_91.png
43_57.png
43_32.png
42_37.png
46_60.png
48_8.png
41_75.png
45_3.png
42_77.png
42_34.png
50_37.png
49_3.png
44_50.png
42_92.png
49_13.png
43_71.png
49_44.png
46_46.png
43_59.png
42_45.png
49_45.png
45_42.png
47_93.png
47_2.png
45_16.png
43_64.png
46_50.png
42_96.png
50_23.png
47_3.png
42_73.png
49_8.png
43_2.png
45_13.png
45_56.png
41_45.png
47_61.png
42_82.png